In [1]:
#Packages used
import pygame
import random

# creates input boxes for getting user input
class InputBox:

    COLOR_INACTIVE = pygame.Color('RED')
    COLOR_ACTIVE = pygame.Color('WHITE')
# Initialises every asepct of the window for the user to see.
    def __init__(self, x, y, w, h, text=''):
        self.rect = pygame.Rect(x, y, w, h)
        self.color = self.COLOR_INACTIVE
        self.text = text
        self.txt_surface = pygame.font.Font(
            None, 32).render(text, True, self.color)
        self.active = False
# Event handler that reads the users input. Handles every input that the user can do.
    def event_handler(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            # If the user clicked on the input_box rect.
            if self.rect.collidepoint(event.pos):
                # Toggle the active variable.
                self.active = not self.active
            else:
                self.active = False
            # Change the current color of the input box.
            self.color = self.COLOR_ACTIVE if self.active else self.COLOR_INACTIVE
            
        if event.type == pygame.KEYDOWN:
            if self.active:
                if event.key == pygame.K_BACKSPACE:
                    self.text = self.text[:-1]
                else:
                    self.text += event.unicode
                # Re-render the text.
                self.txt_surface = pygame.font.Font(
                    None, 32).render(self.text, True, self.color)

#updates the screen once text is set onto the screen
    def update(self):
        # Resize the box if the text is too long.
        width = max(300, self.txt_surface.get_width()+10)
        self.rect.w = width
# Allows for the window to pop up onto the screen.
    def draw(self, screen):
        screen.blit(self.txt_surface, (self.rect.x+5, self.rect.y+15))
        pygame.draw.rect(screen, self.color, self.rect, 2)


# Allows for colors to be chosen in areas of the code. 
# for example: Sets color for user input
class Color:
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    RED = (255, 0, 0)
    BLUE = (0, 128, 255)
    GREEN = (0, 153, 0)
    YELLOW = (255, 255, 0)
    BROWN = (204, 102, 0)
    PINK = (255, 102, 178)
    PURPLE = (153, 51, 255)
    ORANGE = (255,165,0)

    colors = {
        1: WHITE,
        2: YELLOW,
        3: RED,
        4: BLUE,
        5: GREEN,
        6: BLACK,
        7: BROWN,
        8: PINK,
        9: PURPLE,
        10: ORANGE,
    }


# creates the grid and handles the drawings on it. This is basically the simulator that
# powers the grid (chess board) and allows for the visualistation to occur.
class Grid:

    def __init__(self, surface, cellSize, labels):
        self.surface = surface
        self.cols = surface.get_width() // cellSize
        self.rows = surface.get_height() // cellSize
        self.cellSize = cellSize
        self.labels = labels
        self.grid = [[() for _ in range(self.cols)] for _ in range(self.rows)]
        self.font = pygame.font.SysFont('arial', 12, False)
# Draws the border of the board 
    def drawRect(self, pos, value, color):
        x, y = pos
        x *= self.cellSize
        y *= self.cellSize

        fill, border = color

        rect = pygame.Rect(x, y, self.cellSize, self.cellSize)
        self.surface.fill(fill, rect)
        pygame.draw.rect(self.surface, border, rect, 1)
# Controls font size in relation to the boards text
        if value:
            font = pygame.font.SysFont(
                'calibri', 60 - 2 * self.rows - len(str(self.rows)) * 5, True)
            text = font.render(str(value), True, Color.WHITE)
            coord = (3 * self.cellSize) // 10
            self.surface.blit(text, (x + coord + 5, y + coord))

        label = str((y + x * self.rows) // self.cellSize + 1)
        text = pygame.font.SysFont('Arial', 12, True).render(
            label, True, Color.BLACK)
        self.surface.blit(text, (x + 5, y + 5))
# Fills in the boards colours and outlines for each cell
    def drawUseRect(self, pos, visited, pointer_color, cell_color):
        for row in range(self.rows):
            x_axis = row * self.cellSize
            for col in range(self.cols):
                y_axis = col * self.cellSize
                key = (row, col)
                if key in visited:
                    self.drawRect(key, visited[key], cell_color)
                else:
                    if self.labels:
                        indent = ' '
                        if col < 10:
                            indent += ' '
                        text = self.font.render(
                            indent + str(col + row * self.rows + 1), True, Color.WHITE)
                        self.surface.blit(text, (x_axis, y_axis + 5))
                    rect = pygame.Rect(
                        x_axis, y_axis, self.cellSize, self.cellSize)
                    pygame.draw.rect(self.surface, Color.GREEN, rect, 1)
    # Fills visited cells.
        self.drawRect(
            pos, 0 if pos not in visited else visited[pos], pointer_color)


pygame 2.1.2 (SDL 2.0.18, Python 3.9.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import os

# Constants
TITLE = "Knight's Tour using Warnsdorff's Algorithm"
WINDOWS_LOCATION = '350,70'
#Default size can be changed here. Although this shouldn't be changed as the default board size is 8x8 for chess.
SIZE = 8
WIDTH = 750
HEIGHT = 750
#This can be changed as it speeds or slows the process up. I have set it to 60 just for quick results and debugging. This should be below 60(preferably 20)
# Although this is technically not needed and we can publish the finished tour instantly, it is better to understand the path that is taken by the knight 
# Through a visualisation and stylisation of the tour.
FPS = 60
CELL_SIZE = 10

#########################  Warnsdorff's rule  #################################################################################################
###  Warnsdorff's rule is a heuristic for finding a single knight's tour.                                                                   ###
###  The knight is moved so that it always proceeds to the square from which the knight will have the fewest onward moves.                  ###
###  When calculating the number of onward moves for each candidate square, we do not count moves that revisit any square already visited.  ###
###############################################################################################################################################

# Checks whether a square is valid and empty or not 
def isvalid(table, pos):
    size = len(table)
    i, j = pos
    return 0 <= i < size and 0 <= j < size and not table[i][j]

#Returns the number of empty squares adjacent using warnsdorff's rule 
def getmoves(pos):
    x, y = pos
    return [(x + 1, y + 2), (x + 1, y - 2), (x + 2, y + 1), (x + 2, y - 1),
            (x - 1, y + 2), (x - 1, y - 2), (x - 2, y + 1), (x - 2, y - 1)]

# Checks if the move is valid and then goes to move that location, this runs after the minimum access fucntion 
def getaccessiblities(table, pos):
    moves = getmoves(pos)
    count = 0
    for move in moves:
        if isvalid(table, move):
            count += 1

    return count

# Checks to see if it's a possible move and if it is then moves to it from the least optimal position so that the next possible position
# are those that get easy accessability to them
def getminimumaccessible(table, pos):
    moves = getmoves(pos)
    minVal = 8
    minPos = ()
    for move in moves:
        if isvalid(table, move):
            value = getaccessiblities(table, move)
            if value < minVal:
                minVal = value
                minPos = move

    return minPos

# Main friver code for the windows to appear (OS USE)
if __name__ == '__main__':
    # setting Pygame window position
    os.environ['SDL_VIDEO_WINDOW_POS'] = WINDOWS_LOCATION

    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption(TITLE)
    clock = pygame.time.Clock()
    running = True
    axisLabels = True

    # Section that controls user input when first running the program. This ultimately controls user validation for the input boxes in the main menu.
    #This updates and then displays to the screen.

    input_box1 = InputBox(100, 100, 250, 50, ' Table Size (default = 8): ')
    input_boxes = [input_box1]

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    running = False
                    break
            for box in input_boxes:
                box.event_handler(event)

        for box in input_boxes:
            box.update()

        screen.fill((30, 30, 30))
        for box in input_boxes:
            box.draw(screen)

        pygame.display.flip()
        clock.tick(60)

    # validating inputs from the user
    try:
        SIZE = int(input_box1.text.split(' ')[-1])
        if not 5 <= SIZE <= 75:
            raise ValueError
    except ValueError:
        SIZE = 8
    # This is the validation of cell size based on the NxN grid chosen.
    CELL_SIZE = HEIGHT // SIZE
    grid = Grid(surface=screen, cellSize=CELL_SIZE, labels=axisLabels)
    hover = False
    running = True

    # Chooses starting position based on a random cell and grid postition.
    # This allows for the potential for closed and open tours to occur sometimes.
    pos = [random.randrange(SIZE), random.randrange(SIZE)]
    x, y = input_box1.text.split(' ')[-1], input_box1.text.split(' ')[-1]

    table = [[0] * SIZE for _ in range(SIZE)]
    pos = tuple(pos)
    last_pos = pos
    value = 1
    visited = {pos: value}
    flag = True

    # choosing random colors
    pointer_color = (Color.colors[random.randrange(
        3, 10)], Color.colors[random.randrange(1, 11)])
    cell_color = (Color.colors[random.randrange(3, 11)],
                  Color.colors[random.randrange(1, 11)])

    # main loop that runs the draw to screen functions and then the flagging function that determines the spot where the knight has visitied and then 
    # flags it as a random color that is within the colors table. This will then run through the min spot avaialable and continue to loop the code until the board is 
    # eventually finished and the tour is completed.

    while running:
        clock.tick(FPS)
        for event in pygame.event.get():
            # check for closing window
            if event.type == pygame.QUIT:
                running = False
    
        if flag:
            x, y = pos
            last_pos = pos
            table[x][y] = value
            visited.update({pos: value})
            value += 1
            pos = getminimumaccessible(table, pos)
            flag = False

            if value > SIZE ** 2:
                screen.fill(Color.WHITE)
                grid.drawUseRect(last_pos, visited, pointer_color, cell_color)
                pygame.display.flip()
    # This is the section that controls the white spaces on the board if they are not visited, once they are visited then the code will loop so that it 
    # changes the code to the colour.
        if pos and not flag:

            screen.fill(Color.WHITE)
            grid.drawUseRect(last_pos, visited, pointer_color, cell_color)
            pygame.display.flip()

            last_x, last_y = last_pos
            x, y = pos

            if last_x < x:
                last_x += 1
            elif last_x > x:
                last_x -= 1
            else:
                if last_y < y:
                    last_y += 1
                elif last_y > y:
                    last_y -= 1
                else:
                    flag = True

            last_pos = (last_x, last_y)

    print(value)
    pygame.quit()


32
